In [1]:
import pandas as pd
import numpy as np
import email
import os
from email.parser import Parser
import re

In [2]:
# extract the body of email
def walkMsg(msg):
    """
    Input: string of email
    Output: body of email
    """
    for part in msg.walk():
        if part.get_content_type() == "multipart/alternative":
            continue
        yield part.get_payload(decode=1)

In [8]:
# extract the information of email
# still need some works to get number of cc and bcc
def info_of_email(path,List_of_files,classification):
    """
    Input: path, list of files name, classification(spam or ham)
    Ourput: a list with info of email
    """
    parser = Parser()
    info = []
    for files in List_of_files:
        emailText =  open(path + files, 'rb').read()
        email = parser.parsestr(emailText)
        if email.get('cc') == None:
            no_cc = 0
        else:
            no_cc = len(email.get('cc'))
        if email.get("bcc") == None:
            no_bcc = 0
        else:
            no_bcc = len(email.get('bcc'))
        Date = email.get("Date")
        try:
            Date = re.split(':|,| ',Date)
            info.append([Date[4], Date[3], Date[2], Date[0], Date[5], email.get("From"), 
                         email.get('To'), no_cc, no_bcc, 
                         str(email.get("Content-Type")).split(";")[0],
                         email.get('subject'), " ".join([i for i in walkMsg(email) if i != None]),
                         classification])
        except:
            info.append([[], [], [], [], [], email.get("From"), 
                         email.get('To'), no_cc, no_bcc, 
                         str(email.get("Content-Type")).split(";")[0],
                         email.get('subject'), " ".join([i for i in walkMsg(email) if i != None]),
                         classification])
    return info

In [9]:
listing = os.listdir('./Spam_BG/')
listing = [i for i in listing if i[-3:] == "txt"]
email_df = pd.DataFrame(info_of_email('./Spam_BG/', listing, 'spam'),
                       columns=['Year','Month','Day','Weekday',"Hour",
                                'From','To','cc','bcc','Content_type','Subjects','Content','Classification'])

In [10]:
email_df.head()

,Year,Month,Day,Weekday,Hour,From,To,cc,bcc,Content_type,Subjects,Content,Classification
0,2004,Jul,31,Sat,12,"""Bertha "" <Denny@mailbox.sk>",rait@bruce-guenter.dyndns.org,0,0,multipart/alternative,Get it up and keep it up praecox,<html><head><meta http-equiv=Content-Type cont...,spam
1,2004,Jul,31,Sat,22,"""LZEDEMCNODQBEZQBOV@encryption.com"" <Opal.Cham...",cvs@bruce-guenter.dyndns.org,29,0,multipart/alternative,Winner: $332573,<html>\nYour [m]ortgage application was approv...,spam
2,2004,Aug,01,Sun,01,Connie Wong <tatypin@huhmail.com>,Rait <rait@bruce-guenter.dyndns.org>,0,0,multipart/alternative,Need software? Click here.,TOP quality software:<br><br>\n<b>Special Offe...,spam
3,2004,Jul,30,Fri,17,shark@promotions.com <shark@promotions.com>,"""bruceg@em.ca"" <bruceg@em.ca>",0,0,multipart/alternative,Loyalty Bonuses for Shark Casino Players!,\n \n \n \n \n \n Featuring this weeken...,spam
4,2004,Jul,31,Sat,06,"""Important! Search Engine Alert!"" <rockwelldat...","""Bruceg"" <bruceg@em.ca>",0,0,multipart/alternative,Your web site has dropped out of the Search En...,Rockwell Data Corp \n\nThis Email has been sen...,spam
